In [15]:
%pylab inline
import scipy
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, block_diag, identity, hstack
#import matplotlib as mpl
import matplotlib.pyplot as plt
from pyiga import bspline, vform,  geometry, vis, solvers, utils, topology, algebra
from pyiga import assemble
#from patchmesh import *
from sksparse.cholmod import cholesky
#from patchmesh3D import *
#from multipatch import *
from pyiga import adaptive as adap
import itertools
import time

np.set_printoptions(linewidth=100000)
np.set_printoptions(precision=5)
np.set_printoptions(formatter={'float_kind':"{:.3f}".format})

Populating the interactive namespace from numpy and matplotlib


/home/wolfman/anaconda3/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['hstack', 'cholesky', 'identity']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [16]:
kvs=[2*(bspline.make_knots(2,0.0,1.0,3),)]
geos=[geometry.unit_square()]
M=topology.PatchMesh(patches = list(zip(kvs,geos)))

In [20]:
kvs = kvs[0]

In [21]:
kvs

(KnotVector(array([0.000, 0.000, 0.000, 0.333, 0.667, 1.000, 1.000, 1.000]), 2),
 KnotVector(array([0.000, 0.000, 0.000, 0.333, 0.667, 1.000, 1.000, 1.000]), 2))

In [ ]:
kvsx=(kvs[0],kvs[1].deriv(deriv=1)

In [17]:
MP=assemble.Multipatch(M,automatch=True)

Basis setup took 0.0048313140869140625 seconds


In [18]:
u_=MP.L2projection(lambda x,y: (x-0.5)**2 + (y-0.5)**2)

In [19]:
u = bspline.BSplineFunc(kvs[0],u_)

In [9]:
# kvy = [8,2,8,5,8,4]
# kvx = [2,4,3,6,5,8]

# kvs=[(bspline.make_knots(1,0.0,1.0,xN),bspline.make_knots(1,0.0,1.0,yN)) for xN,yN in zip(kvx,kvy)]
# geos=[geometry.unit_square(support=((0.,1.),(0.0,0.5))).scale((2,1)),
#       geometry.unit_square(support=((0.,1.),(0.5,1.))).scale((2,1)),
#       geometry.Quad(np.array([[0.,2.,1.,2.],
#                               [1.,1.,2.,2.]])),
#       geometry.Quad(np.array([[2.,3.,2.,3.5],
#                               [1.,1.,2.,2.5]])),
#       geometry.Quad(np.array([[2.,3.,3.,3.5],
#                               [1.,0.,1.,0.5]])),
#       geometry.Quad(np.array([[2.,2.5,2.,3.],
#                               [0.,-0.5,1.,0.]])),]

# patches = [(tuple(bspline.mapto(k[d],lambda x: g.support[d][1]*x + g.support[d][0]*(1-x)) for d in range(2)), g) for k, g in zip(kvs,geos)]
# M=topology.PatchMesh(patches)
# M.sanity_check()
# #M.h_refine(patches={5:-1, 2:-1});

In [2]:
def drawSkeleton(MP):
    for (p,b,_),(_,_,_),_ in MP.intfs:
        ax,sd=assemble.int_to_bdspec(b)
        geo=MP.mesh.geos()[p]
        kvs=MP.mesh.kvs()[p]
        grid=np.linspace(geo.support[(ax+1)%2][0], geo.support[(ax+1)%2][1],kvs[(ax+1)%2].numdofs)
        if (p,b) == (4,1) or (p,b)==(5,3): grid=grid[1:-1]
        if ax==0:
            Z = geo.eval(grid,np.array([geo.support[0][sd]]))
        else:
            Z = geo.eval(np.array([geo.support[1][sd]]),grid)
        plt.scatter(Z[:,:,0].ravel(),Z[:,:,1].ravel(),color='red',s=15,zorder=3)
    
def drawInterior(MP,r=1):
    for p in range(MP.numpatches):
        kvs, geo = MP.mesh.kvs()[p], MP.mesh.geos()[p]
        gridx=np.linspace(geo.support[0][0],geo.support[0][1] ,kvs[0].numdofs)
        gridy=np.linspace(geo.support[1][0],geo.support[1][1],kvs[1].numdofs)
        Z = geo.eval(gridx,gridy)
        if r != 0:
            plt.scatter(Z[r:-r,r:-r,0].ravel(),Z[r:-r,r:-r,1].ravel(),color='black',s=15)
        else:
            for idx in MP.mesh.outer_boundaries:
                if (p,2) not in MP.mesh.outer_boundaries[idx]: Z = Z[1:,:,:]
                if (p,3) not in MP.mesh.outer_boundaries[idx]: Z = Z[:-1,:,:]
                if (p,0) not in MP.mesh.outer_boundaries[idx]: Z = Z[:,1:,:]
                if (p,1) not in MP.mesh.outer_boundaries[idx]: Z = Z[:,:-1,:]
            plt.scatter(Z[:,:,0].ravel(),Z[:,:,1].ravel(),color='black',s=15,zorder=2)